In [40]:
import wandb
import pandas as pd

In [41]:
from ydata_profiling import ProfileReport

In [42]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

In [43]:
df.shape

(20000, 16)

In [44]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [46]:
# last_review is a a date but datatype is string
df["last_review"]

0        2019-05-26
1               NaN
2        2018-09-19
3        2019-05-24
4        2019-06-23
            ...    
19995    2016-08-27
19996    2019-05-21
19997    2019-05-23
19998    2019-07-01
19999    2019-04-28
Name: last_review, Length: 20000, dtype: object

In [47]:
df.shape

(20000, 16)

In [48]:
df["name"].value_counts()

name
Brooklyn Apartment                            7
Hillside Hotel                                7
Home away from home                           6
New york Multi-unit building                  6
Private Room                                  6
                                             ..
Private Backyard! Beautiful 1BR apartment.    1
Geometric Getaway                             1
A Hidden Gem in Queens                        1
Cozy Studio in Midtown                        1
Private Bedroom in Williamsburg Apt!          1
Name: count, Length: 19768, dtype: int64

In [49]:
# missing values in last_review and reviews_per_month

In [50]:
df.isna().sum()

id                                   0
name                                 7
host_id                              0
host_name                            8
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       4123
reviews_per_month                 4123
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [51]:
# outliers in price

In [52]:
df["price"].describe()

count    20000.000000
mean       153.269050
std        243.325609
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64

In [53]:
(df["price"]>9999).sum()

1

In [54]:
df["price"].loc[df["price"]<=9999].describe()

count    19999.000000
mean       152.776689
std        233.155913
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       9999.000000
Name: price, dtype: float64

In [55]:
df[df["price"]>350].shape

(994, 16)

In [56]:
df.shape

(20000, 16)

In [57]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [58]:
df.shape

(19001, 16)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  number_

In [70]:
run.finish()

In [1]:
import pandas as pd

In [2]:
path_1 = "/home/kbhaskar/Github_Repos/build-ml-pipeline-for-short-term-rental-prices/components/get_data/data/sample1.csv"
df = pd.read_csv(path_1)
#path_2 = "/home/kbhaskar/Github_Repos/build-ml-pipeline-for-short-term-rental-prices/components/get_data/data/sample2.csv"

In [8]:
df.duplicated().sum()

0

In [2]:
#df1

In [3]:
#df1.dtypes

In [4]:
#df1.isna().sum()

In [5]:
#df1.head()

In [60]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaT,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8


In [67]:
df["neighbourhood_group"].value_counts()

neighbourhood_group
Brooklyn         8046
Manhattan        8031
Queens           2331
Bronx             434
Staten Island     159
Name: count, dtype: int64

In [69]:
df["neighbourhood"].value_counts()

neighbourhood
Williamsburg          1526
Bedford-Stuyvesant    1478
Harlem                1086
Bushwick               978
Upper West Side        734
                      ... 
Bay Terrace              1
Howland Hook             1
Spuyten Duyvil           1
South Beach              1
Olinville                1
Name: count, Length: 215, dtype: int64